In [10]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
# sys.path

In [11]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate

In [12]:
%load_ext autoreload
%autoreload 2
%aimport optiml.queries
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from optiml.connection import SnowflakeConnConfig
from optiml.queries import SNFLKQuery
connection = SnowflakeConnConfig(accountname='VUJSGPR-XG91870', warehousename="XSMALL_WH").create_connection()
qlib = SNFLKQuery(connection, 'KIV')

/Users/skumar/.snowsql/config
Connecting...


# Total cost breakdown

## Visualization

In [14]:
df = qlib.total_cost_breakdown('2022-10-01', '2022-10-31')
df

,cost_category,credits,dollars
0,Storage,0.000000,84.288657
1,Compute,1209.029722,2418.059444
2,Cloud Service,46.445960,92.891919
3,Autoclustering,0.000000,0.000000
4,Materialization Views,0.000000,0.000000
5,Replication,0.000000,0.000000
6,Search Optimization,0.000000,0.000000
7,Snowpipe,0.000000,0.000000


In [15]:
df.dtypes

cost_category     object
credits          float64
dollars          float64
dtype: object

In [16]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "table"},{"type": "pie"}]]
)
# fig = go.Figure()

fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['dollars'].tolist()),row=1, col=2)
fig.add_trace(go.Table(header=dict(values=['cost_category','credits', 'dollars'],font_size=10),
                  cells=dict(values=[df['cost_category'],df['credits'],df['dollars']],font_size=10)),row=1, col=1)
#fig.update_layout(font_size=12)
fig.update_layout(
    title={
        'text': "Breakdown of total cost by usage category",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## Analysis
- Compute forms overwhelming majority of the cost
- Cloud services are in aggregate < 10 % of compute so are not an added expense
- Autoclustering used to be on in 2020 but isn't anymore
- Should explore if there are any compute efficiencies to be gained by 
    - Autoclustering
    - Materialized views / caching
    - Search optimization
    - Query optimizations / reducing scans

* Largely used by compute
* In 2020 auto clustering was on but was turned off - why?
* Storage credits at 0 since storage is charged as a flat rate - are numbers consistent?
* Definitely did not use $80K worth of credits in October since total bill is $50K - so something is wrong!


In [12]:
df = qlib.cost_of_compute_ts('2022-10-12', '2022-10-13')

In [13]:
df

,WAREHOUSE_GROUP_NAME,TOTAL_CREDITS_USED,CUMULATIVE_CREDITS_TOTAL,START_TIME,END_TIME
0,WH Compute,10.491666666,10.491666666,2022-10-12 00:00:00-07:00,2022-10-12 01:00:00-07:00
1,WH Compute,10.686666666,21.178333332,2022-10-12 01:00:00-07:00,2022-10-12 02:00:00-07:00
2,WH Compute,9.407499999,30.585833331,2022-10-12 02:00:00-07:00,2022-10-12 03:00:00-07:00
3,WH Compute,10.334166666,40.919999997,2022-10-12 03:00:00-07:00,2022-10-12 04:00:00-07:00
4,WH Compute,9.422499999,50.342499996,2022-10-12 04:00:00-07:00,2022-10-12 05:00:00-07:00
5,WH Compute,9.425833332,59.768333328,2022-10-12 05:00:00-07:00,2022-10-12 06:00:00-07:00
6,WH Compute,10.378333332,70.146666660,2022-10-12 06:00:00-07:00,2022-10-12 07:00:00-07:00
7,WH Compute,9.438333333,79.584999993,2022-10-12 07:00:00-07:00,2022-10-12 08:00:00-07:00
8,WH Compute,9.429999999,89.014999992,2022-10-12 08:00:00-07:00,2022-10-12 09:00:00-07:00
9,WH Compute,26.966944446,115.981944438,2022-10-12 09:00:00-07:00,2022-10-12 10:00:00-07:00


In [14]:
df['TOTAL_CREDITS_USED'].sum()

Decimal('246.493055550')